In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random



In [ ]:
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
books['Image-URL-M'][1]

In [ ]:
users.head()

In [ ]:
ratings.head()

In [ ]:
print(books.shape)
print(ratings.shape)
print(users.shape)

In [ ]:
books.isnull().sum()

In [ ]:
users.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

In [ ]:
users.duplicated().sum()

In [ ]:

users.Age.hist(bins=[0, 10, 20, 30, 40, 50, 100])
plt.title('Age Distribution\n')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.boxplot(y='Age', data=users)
plt.title('Find outlier data in Age column')

In [ ]:
users.Location.unique()

In [ ]:
users.Location.nunique()

In [ ]:

for i in users:
    users['Country']=users.Location.str.extract(r'\,+\s?(\w*\s?\w*)\"*$')   

In [ ]:

users.Country.nunique()

In [ ]:
#drop location column
users.drop('Location',axis=1,inplace=True)

In [ ]:
users.head(2)

In [ ]:
users.isnull().sum()

In [ ]:

users['Country']=users['Country'].astype('str')

In [ ]:

a=list(users.Country.unique())
a=set(a)
a=list(a)
a = [x for x in a if x is not None]
a.sort()
print(a)

In [ ]:
users['Country'].replace(['','01776','02458','19104','23232','30064','85021','87510','alachua','america','austria','autralia','cananda','geermany','italia','united kindgonm','united sates','united staes','united state','united states','us'],
                           ['other','usa','usa','usa','usa','usa','usa','usa','usa','usa','australia','australia','canada','germany','italy','united kingdom','usa','usa','usa','usa','usa'],inplace=True)
     

In [ ]:

plt.figure(figsize=(15,7))
sns.countplot(y='Country',data=users,order=pd.value_counts(users['Country']).iloc[:10].index)
plt.title('Count of users Country wise')

In [ ]:
sns.distplot(users.Age)
plt.title('Age Distribution Plot')
     

In [ ]:
plt.figure(figsize=(15,7))
sns.countplot(y='Book-Author',data=books,order=pd.value_counts(books['Book-Author']).iloc[:10].index)
plt.title('Top 10 Authors')

In [ ]:

plt.figure(figsize=(15,7))
sns.countplot(y='Publisher',data=books,order=pd.value_counts(books['Publisher']).iloc[:10].index)
plt.title('Top 10 Publishers')

Popularity Based Recommendation

In [ ]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [ ]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

In [ ]:
from collections import defaultdict

# Initialize defaultdicts to store the sum of ratings and count of ratings for each book title
sum_ratings = defaultdict(int)
count_ratings = defaultdict(int)

# Iterate through the ratings_with_name DataFrame
for index, row in ratings_with_name.iterrows():
    title = row['Book-Title']
    rating = row['Book-Rating']
    
    # Update the sum and count dictionaries
    sum_ratings[title] += rating
    count_ratings[title] += 1

# Calculate the average rating for each book title using list comprehension
avg_ratings = [[title, sum_ratings[title] / count_ratings[title]] for title in sum_ratings]

# Create a new DataFrame from the list
avg_rating_df = pd.DataFrame(avg_ratings, columns=['Book-Title', 'avg_rating'])


In [ ]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

In [ ]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [ ]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [ ]:
popular_df['Image-URL-M'][0]

Collaborative Filtering Based Recommender System


In [ ]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [ ]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)

In [ ]:
similarity_scores.shape

In [ ]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [ ]:
books = books.merge(popular_df[['Book-Title', 'num_ratings', 'avg_rating']], on='Book-Title', how='left')


In [2]:
recommend('1984')

NameError: name 'recommend' is not defined

In [ ]:
pt.index[545]

In [ ]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))


In [ ]:
books.drop_duplicates('Book-Title')

In [ ]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))


In [ ]:
import pandas as pd

# Read CSV files into DataFrames
books = pd.read_csv('books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('users.csv')

In [ ]:
# Preprocess data
# Extract country from location
users['Country'] = users['Location'].apply(lambda x: x.split(',')[-1].strip())

# Merge DataFrames
ratings_users = pd.merge(ratings, users, on='User-ID')
rating_books_users = pd.merge(ratings_users, books, on='ISBN')

In [ ]:

# Group ratings by age, country, and book
grouped_ratings = rating_books_users.groupby(['Age', 'Country', 'ISBN', 'Book-Title', 'Book-Author', 'Image-URL-M']).agg({'Book-Rating': ['count', 'mean']}).reset_index()
grouped_ratings.columns = ['Age', 'Country', 'ISBN', 'Book-Title', 'Book-Author', 'Image-URL-M', 'Count', 'Average-Rating']



In [ ]:
# Define a function to get recommendations for a given age and country
def get_recommendations(age, country, num_recommendations=5):
    filtered_ratings = grouped_ratings[(grouped_ratings['Age'] == age) & (grouped_ratings['Country'] == country)]
    sorted_ratings = filtered_ratings.sort_values(by='Average-Rating', ascending=False)
    top_recommendations = sorted_ratings.head(num_recommendations)
    return top_recommendations

In [ ]:
# Test the function with sample input
age = 23
country = 'germany'
recommendations = get_recommendations(age, country)
print(recommendations)

In [ ]:
import pickle

with open('grouped_ratings.pkl', 'wb') as f:
    pickle.dump(grouped_ratings, f)
